<a href="https://colab.research.google.com/github/AshbeeKim/Kaggle_task/blob/main/test_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<06.04(금) 14:00 ~ 15:00 텍스트 분석>
Data set : Mercari Price Suggestion Challenge(https://www.kaggle.com/c/mercari-price-suggestion-challenge/overview)

1. [중] 대/중/소 형식으로 개별 컬럼을 생성하시오.
    * category name 을 대/중/소 형식으로 개별 컬럼으로 생성하고 중복을 제거 한 다음 코드를 첨부하시오.
2.  [중]텍스트를 tokenize를 하시오.
3.  category_name 변수에서 가장 많이 있는 품목을 코드로 밝히시오(plotly를 사용하여서)
4. TF-IDF를 사용하시오.
5. 모델을 적용해보고 결과를 해석하시오.

# import & download

In [ ]:
# import os  

# from google.colab import files
# files.upload()

# !mkdir -p ~/.kaggle

# !mv kaggle.json ~/.kaggle
# #!cd kaggle.json ~/.kaggle

# !chmod 600 ~/.kaggle/kaggle.json
# !cd ~/.kaggle  

# %cd ~/.kaggle

# !ls -1ha ~/.kaggle/kaggle.json

# !export KAGGLE_USERNAME=ashbeekim
# !export KAGGLE_KEY=xxxxxxxxxxxxxx

Saving kaggle.json to kaggle.json
mv: 'kaggle.json' and '/root/.kaggle/kaggle.json' are the same file
/root/.kaggle
/root/.kaggle/kaggle.json


In [ ]:
import os  
os.environ['KAGGLE_CONFIG_DIR'] = "/root/.kaggle"

from google.colab import drive  
ROOT = "/content/drive"     # 드라이브 기본 경로
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)           # 드라이브 기본 경로 Mount

from os.path import join  

MY_GOOGLE_DRIVE_PATH = 'MyDrive/Task/'
PROJECT = "Mercari_Price"
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH, PROJECT)

# !mkdir "{PROJECT_PATH}"
# print(PROJECT_PATH)

%cd "{PROJECT_PATH}"

/content/drive
Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1srYR8TqwFsUSR4BYzQtIBmWUKrEBjd44/Task/Mercari_Price


In [ ]:
# # !kaggle competitions list
# # !kaggle datasets list
# # !kaggle datasets list -s Movies
# # !kaggle competitions download -c house-prices-advanced-regression-techniques
# # !kaggle datasets download -d percevalw/englishfrench-translations  
# # !unzip englishfrench-translations.zip  
# # !ls
# # !kaggle competitions list -s mercari
# !kaggle competitions download -c mercari-price-suggestion-challenge

# !7z e train.tsv.7z
# !7z e test.tsv.7z
# !7z e sample_submission.csv.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 77912192 bytes (75 MiB)

Extracting archive: train.tsv.7z
--
Path = train.tsv.7z
Type = 7z
Physical Size = 77912192
Headers Size = 122
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      3% - train.tsv                  7% - train.tsv                 11% - train.tsv                 14% - train.tsv                 18% - train.tsv                 22% - train.tsv                 26% - train.tsv                 29% - train.tsv                 32% - train.tsv

# read datas

In [ ]:
!ls

sample_submission.csv		test_stg2.tsv.zip  train.tsv
sample_submission.csv.7z	test.tsv	   train.tsv.7z
sample_submission_stg2.csv.zip	test.tsv.7z


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

In [ ]:
import csv
data = pd.read_csv(f'{PROJECT_PATH}/train.tsv', sep='\t')
tst_data = pd.read_csv(f'{PROJECT_PATH}/test.tsv', sep='\t')
sS_data = pd.read_csv(f'{PROJECT_PATH}/sample_submission.csv')

In [ ]:
origin_dt_lst = ['data','tst_data','sS_data']

In [ ]:
for lst in origin_dt_lst:
  print(f"{lst}'s shape : {globals()[lst].shape}")

data's shape : (1482535, 8)
tst_data's shape : (693359, 7)
sS_data's shape : (693359, 2)


In [ ]:
for lst in origin_dt_lst:
  print(f"{lst}'s col : {globals()[lst].columns}")

data's col : Index(['train_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'price', 'shipping', 'item_description'],
      dtype='object')
tst_data's col : Index(['test_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'shipping', 'item_description'],
      dtype='object')
sS_data's col : Index(['test_id', 'price'], dtype='object')


In [ ]:
for lst in origin_dt_lst:
  print(f"\n\n{lst}'s info : \n")
  print(f"{locals()[lst].info()}")



data's info : 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   train_id           1482535 non-null  int64  
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null  int64  
 3   category_name      1476208 non-null  object 
 4   brand_name         849853 non-null   object 
 5   price              1482535 non-null  float64
 6   shipping           1482535 non-null  int64  
 7   item_description   1482531 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 90.5+ MB
None


tst_data's info : 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   test_id            693359 non-null  int64 
 1   name               69

결측치 현재로는 안보이고, target은 price임

# init

In [ ]:
train = data.copy()
test = tst_data.copy()

train = pd.concat([train.drop(['price'], axis=1),train['price']], axis=1)
test = pd.concat([test, sS_data.iloc[:,1]], axis=1)

현재까지는 train과 test의 shape 맞춤



```
# Use `iloc[]` to select a row
display(df.iloc[0])
display(df.loc[0])
display(df.ix[0])

# Use `loc[]` to select a column
display(df.loc[:,'A'])
display(df['A'])

# 특정 row, column을 선택하기
display(df.ix[0]['A'])
display(df.loc[0]['B'])


출처: https://3months.tistory.com/292 [Deep Play]
```



In [ ]:
check_dt = ['train', 'test']

In [ ]:
from IPython.display import display

for dt in check_dt:
  print(f"\n[5 of {dt}]")
  print(f"{display(globals()[dt].head())}")


[5 of train]


,train_id,name,item_condition_id,category_name,brand_name,shipping,item_description,price
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,1,No description yet,10.0
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,0,This keyboard is in great condition and works ...,52.0
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,1,Adorable top with a hint of lace and a key hol...,10.0
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,1,New with tags. Leather horses. Retail for [rm]...,35.0
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,0,Complete with certificate of authenticity,44.0


None

[5 of test]


,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description,price
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7,26.738
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined...",26.738
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...,26.738
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...,26.738
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...,26.738


None


안보였던 결측치가 있음....brand_name에 np.nan으로 들어갔는데 해당 컬럼의 type이 str라서 확인이 불가했나봄

In [ ]:
for dt in check_dt:
  print(f"{dt}'s describe : \n{globals()[dt].describe().round(3)}\n")

train's describe : 
          train_id  item_condition_id        price     shipping
count  1482535.000        1482535.000  1482535.000  1482535.000
mean    741267.000              1.907       26.738        0.447
std     427971.135              0.903       38.586        0.497
min          0.000              1.000        0.000        0.000
25%     370633.500              1.000       10.000        0.000
50%     741267.000              2.000       17.000        0.000
75%    1111900.500              3.000       29.000        1.000
max    1482534.000              5.000     2009.000        1.000

test's describe : 
          test_id  item_condition_id    shipping       price
count  693359.000         693359.000  693359.000  693359.000
mean   346679.000              1.906       0.448      26.738
std    200155.647              0.903       0.497       0.000
min         0.000              1.000       0.000      26.738
25%    173339.500              1.000       0.000      26.738
50%    346679.000 

In [ ]:
for dt in check_dt:
  print(f"{dt}'s correlation : \n{globals()[dt].corr().round(3)}\n")

train's correlation : 
                   train_id  item_condition_id  price  shipping
train_id                1.0             -0.000  0.000     0.000
item_condition_id      -0.0              1.000 -0.001    -0.191
price                   0.0             -0.001  1.000    -0.097
shipping                0.0             -0.191 -0.097     1.000

test's correlation : 
                   test_id  item_condition_id  shipping  price
test_id              1.000             -0.002     0.001    0.0
item_condition_id   -0.002              1.000    -0.189    0.0
shipping             0.001             -0.189     1.000   -0.0
price                0.000              0.000    -0.000    1.0



1. [중] 대/중/소 형식으로 개별 컬럼을 생성하시오.
    * category name 을 대/중/소 형식으로 개별 컬럼으로 생성하고 중복을 제거 한 다음 코드를 첨부하시오.
2.  [중]텍스트를 tokenize를 하시오.
3.  category_name 변수에서 가장 많이 있는 품목을 코드로 밝히시오(plotly를 사용하여서)
4. TF-IDF를 사용하시오.
5. 모델을 적용해보고 결과를 해석하시오.

In [ ]:
def checking_IQR(df, col):
  q25 = np.percentile(df[col].values, 25)
  q50 = np.percentile(df[col].values, 50)
  q75 = np.percentile(df[col].values, 75)
  return q25, q50, q75

def outlier_replace(df, col,w=1.5):
  q25 = np.percentile(df[col].values, 25)
  q75 = np.percentile(df[col].values, 75)

  IQR = q75 - q25
  l_est = df[col][df[col]<=(q25 - (IQR*w))].values
  h_est = df[col][df[col]>=(q75 + (IQR*w))].values

  df[col] = df[col].replace(l_est,q25)
  df[col] = df[col].replace(h_est,q75)

def outlier_drop(df, col, w=1.5):
  q25 = np.percentile(df[col].values, 25)
  q75 = np.percentile(df[col].values, 75)

  IQR = q75 - q25
  l_est = q25 - (IQR*w)
  h_est = q75 + (IQR*w)

  o_idx = df[col][(df[col]<l_est)|(df[col]>h_est)].index
  df = df.drop(o_idx, axis=0, inplace=True)
  return df

def binning_IQR(df, col):
  q25 = np.percentile(df[col].values, 25)
  q50 = np.percentile(df[col].values, 50)
  q75 = np.percentile(df[col].values, 75)

  belowq25 = df[col][df[col]<q25].values
  q25toq50 = df[col][(df[col]>=q25)&(df[col]<q50)].values
  q50toq75 = df[col][(df[col]<q75)&(df[col]>=q50)].values
  aboveq75 = df[col][df[col]>=q75].values

  df[col] = df[col].replace(belowq25,1)
  df[col] = df[col].replace(q25toq50,2)
  df[col] = df[col].replace(q50toq75,3)
  df[col] = df[col].replace(aboveq75,4)

  df[col] = df[col].astype('int64')
  return df

# def binning_age(df, col):
#   kidsteens = df[col][df[col]<20].values
#   s20tos30 = df[col][(df[col]>=20)&(df[col]<40)].values
#   s40tos50 = df[col][(df[col]<60)&( df[col]>=40)].values
#   seniors = df[col][df[col]>=60].values

#   df[col] = df[col].replace(kids,1)
#   df[col] = df[col].replace(s20tos30,2)
#   df[col] = df[col].replace(s40tos50,3)
#   df[col] = df[col].replace(senior,4)

#   df[col] = df[col].astype('int64')  

def all_cont_catg_col(data):
  all_col = [cols for cols in train.columns]

  print(f'Columns : {len(all_col)}')

  cont_col = [cols for cols in data.columns[data.dtypes != 'object']]
  catg_col = [cols for cols in data.columns[data.dtypes == 'object']]

  return all_col, cont_col, catg_col

def print_all_cont_catg(data, all_col, cont_col, cat_col):
  print(f'\t<<<All Columns>>> : \n{all_col}')
  print('\t<<<Continuous>>>')
  for check in cont_col:
    print(f'[{check} IQR] : {checking_IQR(data, check)} \n[{check} max] : {data[check].max()} [{check} min] : {data[check].min()}')
  print('\n\t<<<Categorical>>>')
  for check in cat_col:
    try:
      print(f'[{check} unique] : {data[check].unique()} \n[{check} max] : {data[check].max()} [{check} min] : {data[check].min()}')
    except:
      print(f'[{check} unique] : This column has "None"')  

def dtp_feat(data):
    num_col = data.columns[data.dtypes != 'object']
    cat_col = data.columns[data.dtypes == 'object']
    
    n_flen = len(num_col)
    c_flen = len(cat_col)
    
    num_feat = [num_col[x] for x in range(n_flen)]
    cat_feat = [cat_col[x] for x in range(c_flen)]
    return num_feat, cat_feat

def zscr_idx(data, col, tres=3):
    mean = data[col].mean()
    std = data[col].std()
    
    otrl = []
    otrl_idx = []

    for idx, val in enumerate(data[col]):
    #for val, idx in zip(data[col], data.index):
        if ((val-mean)/std>tres)|((val-mean)/std<-tres):
            otrl.append(val)
            otrl_idx.append(idx)
    #otrl = [val for val in data[col] if ((val-mean)/std>3)|((val-mean)/std<-3)]
    return otrl, otrl_idx

def del_outlier1(df,o_idx):
    df = df.drop(o_idx, axis=0, inplace=True)
    return df

def del_outlier2(df,o_idx):
    df = df.drop(o_idx, axis=0)
    return df

# def re_index(df):
    # re_idx = [idx for idx in range(df.shape[0])]
    #re_df = df.reindex(re_idx) #재배열임...
    # return re_df  

def outlier_box_dist(data, target_col):
  fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,8), constrained_layout=True)
  fig.suptitle('Checking Outlier using Boxplot and Displot')
  #boxplot
  sns.boxplot(data[target_col], ax=ax[0], palette='crest')
  #distplot
  sns.distplot(data[target_col], ax=ax[1], rug=True, hist=False, color='y')
  plt.show()

def outlier_boxplot_byT(df, target_col, col=2):
  wthoutT = df.copy().drop(target_col,axis=1)
  n = len(wthoutT.columns)
  row = math.ceil(n/col)
  f, ax = plt.subplots(row, col, figsize=(12,16), constrained_layout=True)

  for i, col in enumerate(wthoutT.columns):
    sns.boxplot(data=df, x=target_col, y=col, ax=ax[i//col,i%col], palette='vlag_r')
  plt.show() 

def all_displot_df(df, col=2):
  n = df.shape[1]
  row = math.ceil(n/col)

  f, ax = plt.subplots(row, col, figsize=(16,12), constrained_layout=True)
  for i in range(df.shape[1]):
    # sns.displot(df.iloc[:,i], ax[math.ceil(i/col),i%col], kde=True)
    sns.displot(df.iloc[:,i], ax=ax[i//col,i%col], kde=True)
    plt.title(df.columns[i])
  plt.show()

# 드롭하고 reset_index()한 뒤, object인 열을 기준으로 row에 날아간 데이터가 있는지 확인
def chk_nan_from_object(data, col):
  idx_lst = []
  for idx, chk in enumerate(data[col]):
    if type(chk) == float:
      idx_lst.append(idx)
  return data.iloc[idx_lst]


def modified_zscr(data, col, k=1.4826):
    med = data[col].median()
    
    #median absolute deviation
    m_a_d = data[col].map(lambda x: np.abs(x-med)).median()

    mod_zscr=[]
    for x in data[col]:
      mod_zscr.append(k/2*(x-med)/m_a_d)
    #mod_zscr = list(data[col].map(lambda x: k/2*(x-med)/m_a_d))
    return mod_zscr

def mod_zscr_idx(data, col, mod_zscr, tres=3):
    mod_otrl = []
    mod_otrl_idx = []
    
    for scr, idx, val in zip(mod_zscr, data.index, data[col]):
        if (scr > tres)|(scr < -tres):
            mod_otrl.append(val)
            mod_otrl_idx.append(idx)

    return mod_otrl, mod_otrl_idx

def mapping_dict(data, key_col, val_col, new_col): #key_col: int -> #asec #key_col : str->가끔 미싱values 있을 때도 있음 #value가 출력됨
  kv_dict = {key: value for key, value in zip(data[key_col].unique(), data[val_col].unique())}
  data[new_col] = data[key_col].map(kv_dict)
  return data
def drop_col(data, col_lst):
  data = data.drop(col_lst, axis=1)
  return data

def df_val_dict(data, key_col, val_col, new_col):
  kv_dict = {key: value for key, value in zip(data[key_col].values, data[val_col].values)}
  data[new_col] = data[val_col].map(kv_dict)
  return data

def df_meridiem(data, hr_col, min_col, sec_col):
  rep_lst = []

  for h,m,s in zip(data[hr_col], data[min_col], data[sec_col]):
    if h>=0 and h<12:
      if m>=0 and m<60:
        if s>=0 and s<60:
          rep_lst.append('AM')
    else:
      rep_lst.append('PM')
  meridiem = pd.DataFrame(rep_lst, columns=['meridiem'])
  return pd.concat([data, meridiem], axis=1)

def srch_holiday(data, conds, y_col, m_col, d_col):
  h_day = []

  for y,m,d in zip(data[y_col], data[m_col], data[d_col]):
    if date(y, m, d) in conds:
      h_day.append(conds.get(date(y,m,d)))
    else:
      h_day.append('non-holiday')
    # data['holiday'] = pd.DataFrame(h_day)
  return pd.DataFrame(h_day)

# return전에 pd.DataFrame()짜서, 출력하는 것과 return뒤에 바로 출력하는 것의 큰 속도 차이는 없지만, 그래도 출력이 된 것을 다시 return 하는 것보다 return에서 바로 출력하는 것이 빠름

#wd_col은 딕셔너리 형태로 된 pd.Series임
#현재 flow에서는 wd_col='pu_dt_w'
def srch_businessday(data, wd_col, hd_col):
  bs_day = []

  for wd, hd in zip(data[wd_col].values, data[hd_col]):
    if (wd!='Saturday') & (wd!='Sunday') & (hd=='non-holiday'):
      bs_day.append('Businessday')
    else:
      bs_day.append('Dobby is Free!!!')
  return pd.DataFrame(bs_day)

def geomtry_loc(data, lon_col, lat_col):
  gdf = geopandas.GeoDataFrame(data, geometry=geopandas.points_from_xy(data.lon_col, data.lat_col))
  return gdf

def location(data, lat_col, lon_col):
    point = []
    
    for lat, lon in zip(data[lat_col], data[lon_col]):
        point.append([lat, lon])
        
    return point

def dplct_loc(data, st_lat_col, st_lon_col, ed_lat_col, ed_lon_col):
    st_loc =[]
    ed_loc = []
    
    for slat, slon, elat, elon in zip(data[st_lat_col], data[st_lon_col], data[ed_lat_col], data[ed_lon_col]):
        st_loc.append([slat, slon])
        ed_loc.append([elat, elon])
        #tuple?list?
        
    return st_loc, ed_loc

def model_choose(lst, num=0):
    model = lst[num]
    return model

def select_features(X_train, y_train, X_test, n_est=42):
    # configure to select a subset of features
    feat_s = SelectFromModel(RandomForestClassifier(n_estimators=n_est), max_features=5)
    # learn relationship from training data
    feat_s.fit(X_train, y_train)
    # transform train input data
    X_trn_fs = feat_s.transform(X_train)
    # transform test input data
    X_tst_fs = feat_s.transform(X_test)
    return X_trn_fs, X_tst_fs, feat_s

In [ ]:
trn_alst, trn_nlst, trn_clst = all_cont_catg_col(train)
tst_alst, tst_nlst, tst_clst = all_cont_catg_col(test)

Columns : 8
Columns : 8


In [ ]:
print_all_cont_catg(train, trn_alst, trn_nlst, trn_clst)

	<<<All Columns>>> : 
['train_id', 'name', 'item_condition_id', 'category_name', 'brand_name', 'shipping', 'item_description', 'price']
	<<<Continuous>>>
[train_id IQR] : (370633.5, 741267.0, 1111900.5) 
[train_id max] : 1482534 [train_id min] : 0
[item_condition_id IQR] : (1.0, 2.0, 3.0) 
[item_condition_id max] : 5 [item_condition_id min] : 1
[shipping IQR] : (0.0, 0.0, 1.0) 
[shipping max] : 1 [shipping min] : 0
[price IQR] : (10.0, 17.0, 29.0) 
[price max] : 2009.0 [price min] : 0.0

	<<<Categorical>>>
[name unique] : ['MLB Cincinnati Reds T Shirt Size XL' 'Razer BlackWidow Chroma Keyboard'
 'AVA-VIV Blouse' ... '21 day fix containers and eating plan'
 'World markets lanterns' 'Brand new lux de ville wallet'] 
[name max] : ：）iPhone 6/6s RoseGold Mirror case [name min] : ! 3 girls t shirts
[category_name unique] : This column has "None"
[brand_name unique] : This column has "None"
[item_description unique] : This column has "None"


In [ ]:
trn_none = ['category_name', 'brand_name', 'item_description']
for col in trn_none:
  print(f'{col} : \n',chk_nan_from_object(train, col).count(),'\n')

category_name : 
 train_id             6327
name                 6327
item_condition_id    6327
category_name           0
brand_name           2870
shipping             6327
item_description     6327
price                6327
dtype: int64 

brand_name : 
 train_id             632682
name                 632682
item_condition_id    632682
category_name        629225
brand_name                0
shipping             632682
item_description     632679
price                632682
dtype: int64 

item_description : 
 train_id             4
name                 4
item_condition_id    4
category_name        4
brand_name           1
shipping             4
item_description     0
price                4
dtype: int64 



In [ ]:
print_all_cont_catg(test, tst_alst, tst_nlst, tst_clst)


In [ ]:
trn_col = ['trn_alst', 'trn_nlst', 'trn_clst']
tst_col = ['tst_alst', 'tst_nlst', 'tst_clst']

In [ ]:
#cf)  https://3months.tistory.com/296?category=753896